In [1]:
!pip install ultralytics
!pip install segment-anything
!pip install opencv-python-headless
!pip install matplotlib
!pip install kagglehub
!pip install supervision
!git clone https://github.com/IDEA-Research/GroundingDINO.git
%cd GroundingDINO
!pip install -e .
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!wget https://github.com/IDEA-Research/GroundingDINO/releases/download/v0.1.0-alpha/groundingdino_swint_ogc.pth -P checkpoints

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 896.3/896.3 kB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.5/181.5 kB 2.5 MB/s eta 0:00:00
Cloning into 'GroundingDINO'...
remote: Enumerating objects: 463, done.
remote: Counting objects: 100% (240/240), done.
remote: Compressing objects: 100% (104/104), done.
remote: Total 463 (delta 175), reused 136 (delta 136), pack-reused 223 (from 1)
Receiving objects: 100% (463/463), 12.87 MiB | 31.46 MiB/s, done.
Resolving deltas: 100% (241/241), done.
/content/GroundingDINO
Obtaining file:///content/GroundingDINO
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.8/46.8 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 256.2/256.2 kB 7.3 MB/s eta 0:00:00
  Running setup.py develop for groundingdino
Looking in indexes: https://download.pytorch.org/whl/cu118
--2024-11-27 01:34:26--  https://github.com/IDEA-Research/GroundingDINO/releases/download/v0.1.0-alpha/groundingdi

In [2]:
from google.colab import drive
drive.mount('/content/drive')
base_dir = "/content/drive/MyDrive/Big_data_CNN_201124"

import os
if not os.path.exists(base_dir):
    os.makedirs(base_dir)

print(f"Carpeta base configurada en: {base_dir}")


Mounted at /content/drive
Carpeta base configurada en: /content/drive/MyDrive/Big_data_CNN_201124


In [3]:
import kagglehub
from shutil import copytree

path = kagglehub.dataset_download("sshikamaru/car-object-detection")
target_dir = os.path.join(base_dir, "car-object-detection")
copytree(path, target_dir, dirs_exist_ok=True)
print(f"Dataset descargado en: {target_dir}")

100%|██████████| 112M/112M [00:01<00:00, 110MB/s]

Extracting files...


Dataset descargado en: /content/drive/MyDrive/Big_data_CNN_201124/car-object-detection


In [ ]:
# Redimensionar imágenes
import cv2
from tqdm import tqdm

folders = [
    "/content/drive/MyDrive/Big_data_CNN_201124/car-object-detection/data/training_images",
    "/content/drive/MyDrive/Big_data_CNN_201124/car-object-detection/data/testing_images",
]

output_base_folder = "/content/drive/MyDrive/Big_data_CNN_201124/resized_images"
for input_folder in folders:
    output_folder = os.path.join(output_base_folder, os.path.basename(input_folder))
    os.makedirs(output_folder, exist_ok=True)
    print(f"Procesando imágenes de: {input_folder}")
    for filename in tqdm(os.listdir(input_folder)):
        if filename.endswith((".jpg", ".png")):
            img = cv2.imread(os.path.join(input_folder, filename))
            resized = cv2.resize(img, (128, 128), interpolation=cv2.INTER_AREA)
            cv2.imwrite(os.path.join(output_folder, filename), resized)

print(f"Imágenes redimensionadas guardadas en: {output_base_folder}")

Procesando imágenes de: /content/drive/MyDrive/Big_data_CNN_201124/car-object-detection/data/training_images


100%|██████████| 1001/1001 [00:39<00:00, 25.16it/s]


Procesando imágenes de: /content/drive/MyDrive/Big_data_CNN_201124/car-object-detection/data/testing_images


100%|██████████| 175/175 [00:04<00:00, 36.15it/s]


Imágenes redimensionadas guardadas en: /content/drive/MyDrive/Big_data_CNN_201124/resized_images


In [4]:
# Cargar modelo GroundingDINO
from groundingdino.util.inference import load_model, load_image, predict, annotate

model_config_path = "/content/GroundingDINO/groundingdino/config/GroundingDINO_SwinT_OGC.py"
model_checkpoint_path = "/content/GroundingDINO/checkpoints/groundingdino_swint_ogc.pth"

model = load_model(model_config_path, model_checkpoint_path)
print("Modelo Grounding DINO cargado correctamente")

final text_encoder_type: bert-base-uncased


The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Modelo Grounding DINO cargado correctamente


In [5]:
# Predicción y anotación para visualización
import supervision as sv

In [6]:
import os
from PIL import Image
from tqdm import tqdm
import supervision as sv
from groundingdino.util.inference import load_model, load_image, predict, annotate

# Configuración del modelo
model_config_path = "/content/GroundingDINO/groundingdino/config/GroundingDINO_SwinT_OGC.py"
model_checkpoint_path = "/content/GroundingDINO/checkpoints/groundingdino_swint_ogc.pth"
model = load_model(model_config_path, model_checkpoint_path).to("cpu")  # Forzar a CPU
print("Modelo Grounding DINO cargado correctamente")

# Parámetros de predicción
text = "car"  # Texto para detección
BOX_THRESHOLD = 0.26
TEXT_THRESHOLD = 0.30

# Directorios de entrada y salida
input_folder = "/content/drive/MyDrive/Big_data_CNN_201124/car-object-detection/data/training_images"
output_folder = "/content/drive/MyDrive/Big_data_CNN_201124/detection"
os.makedirs(output_folder, exist_ok=True)

# Obtener las primeras 50 imágenes
all_files = [f for f in os.listdir(input_folder) if f.endswith((".jpg", ".png"))]
first_100_files = all_files[:100]

# Procesar las primeras 50 imágenes
for filename in tqdm(first_100_files):
    if filename.endswith((".jpg", ".png")):
        image_path = os.path.join(input_folder, filename)

        try:
            # Cargar imagen y preprocesar
            image_source, image_tensor = load_image(image_path)

            # Realizar predicción
            boxes, logits, phrases = predict(
                model=model,
                image=image_tensor,
                caption=text,
                box_threshold=BOX_THRESHOLD,
                text_threshold=TEXT_THRESHOLD,
                device="cpu"  # Cambiar a 'cuda' si tienes GPU
            )

            # Anotar la imagen
            annotated_frame = annotate(
                image_source=image_source,
                boxes=boxes,
                logits=logits,
                phrases=phrases
            )

            # Guardar la imagen anotada
            output_image_path = os.path.join(output_folder, filename)
            annotated_pil = Image.fromarray(annotated_frame)
            annotated_pil.save(output_image_path)

        except Exception as e:
            print(f"Error procesando {filename}: {e}")

print(f"Imágenes anotadas guardadas en: {output_folder}")

final text_encoder_type: bert-base-uncased
Modelo Grounding DINO cargado correctamente


100%|██████████| 100/100 [1:03:29<00:00, 38.10s/it]

Imágenes anotadas guardadas en: /content/drive/MyDrive/Big_data_CNN_201124/detection


In [7]:
!pip install segment-anything
!pip install opencv-python-headless
!pip install matplotlib


In [9]:
!wget https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth -P /content/checkpoints

--2024-11-27 03:05:09--  https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 18.244.202.25, 18.244.202.62, 18.244.202.103, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|18.244.202.25|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2564550879 (2.4G) [binary/octet-stream]
Saving to: ‘/content/checkpoints/sam_vit_h_4b8939.pth’

sam_vit_h_4b8939.pt 100%[===================>]   2.39G  19.3MB/s    in 37s     

2024-11-27 03:05:46 (66.1 MB/s) - ‘/content/checkpoints/sam_vit_h_4b8939.pth’ saved [2564550879/2564550879]



In [10]:
import os

checkpoint_path = "/content/checkpoints/sam_vit_h_4b8939.pth"
if os.path.exists(checkpoint_path):
    print("Checkpoint encontrado:", checkpoint_path)
else:
    print("Checkpoint no encontrado. Verifica la ruta.")


Checkpoint encontrado: /content/checkpoints/sam_vit_h_4b8939.pth


In [11]:
from segment_anything import sam_model_registry, SamPredictor
import cv2
import os
from tqdm import tqdm
import numpy as np
from PIL import Image

# Ruta al checkpoint pre-entrenado de SAM
sam_checkpoint = "/content/checkpoints/sam_vit_h_4b8939.pth"  # Cambiar por tu ruta local
model_type = "vit_h"  # Modelo pre-entrenado (vit_h, vit_l o vit_b)

# Cargar el modelo
sam = sam_model_registry[model_type](checkpoint=sam_checkpoint)
predictor = SamPredictor(sam)


In [ ]:
input_folder = "/content/drive/MyDrive/Big_data_CNN_201124/detection"  # Carpeta de imágenes anotadas
output_folder = "/content/drive/MyDrive/Big_data_CNN_201124/masks"  # Carpeta para guardar máscaras
os.makedirs(output_folder, exist_ok=True)

# Procesar cada imagen
for filename in tqdm(os.listdir(input_folder)):
    if filename.endswith((".jpg", ".png")):
        image_path = os.path.join(input_folder, filename)

        # Cargar la imagen
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convertir a RGB para SAM
        predictor.set_image(image)

        # Generar máscara
        masks, scores, _ = predictor.predict(
            point_coords=None,  # Puedes usar puntos específicos para mejorar
            point_labels=None,
            box=None,  # Si tienes cajas anotadas, pásalas aquí
            multimask_output=True
        )

        # Seleccionar la mejor máscara (mayor puntuación)
        best_mask = masks[np.argmax(scores)]

        # Guardar la máscara como imagen
        mask_path = os.path.join(output_folder, f"{os.path.splitext(filename)[0]}_mask.png")
        Image.fromarray((best_mask * 255).astype(np.uint8)).save(mask_path)

print(f"Máscaras generadas y guardadas en: {output_folder}")


 17%|█▋        | 145/854 [5:35:41<27:33:35, 139.94s/it]